In [17]:
import requests
import bs4
import sys
import math
import json


url_list = ['http://www.chinadaily.com.cn/china/2016-04/20/content_24701635.htm',
            'http://www.chinadaily.com.cn/china/2016-04/20/content_24700746.htm',
            'http://www.chinadaily.com.cn/china/2016-04/20/content_24681482.htm',
            'http://www.chinadaily.com.cn/china/2016-04/19/content_24675530.htm',
            'http://www.chinadaily.com.cn/china/2016-04/19/content_24675455.htm',
            'http://www.chinadaily.com.cn/china/2016-04/19/content_24674074.htm',
            'http://www.chinadaily.com.cn/china/2016-04/19/content_24655536.htm',
            'http://www.chinadaily.com.cn/china/2016-04/18/content_24643685.htm',
            'http://www.chinadaily.com.cn/china/2016-04/18/content_24636917.htm',
            'http://www.chinadaily.com.cn/china/2016-04/15/content_24562198.htm'
            ]

articles_title = []
articles_content = []
# 解析
for pos,url in enumerate(url_list):
    r = requests.get(url)
    soup1 = bs4.BeautifulSoup(r.text,'lxml')
    soup2 = bs4.BeautifulSoup(str(soup1.find_all(id="Title_e")),'lxml')
#     print(soup2.h1.string)
    articles_title.append(soup2.h1.string)
#     print(articles_title)
    mystr = ""
    soup3 = bs4.BeautifulSoup(str(soup1.find_all(id="Content")),'lxml')
#     print(soup3)
    for x in soup3.find_all("p"):
#         print(x.string)
        mystr = mystr + x.string
#         print(mystr)
#    初始化
    str_p = ""
    contents = []
# 去掉标点,分词
    for pos,x in enumerate(mystr):
        if x == '.' or x == ',':
            if pos < (len(mystr) - 1) and mystr[pos+1] >= '0' and mystr[pos+1] <= '9':
                str_p = str_p + x
            elif str_p == "":
                continue
            else:
                contents.append(str_p)
                str_p = ""
        elif x == '(' or x == ')' or x == ' ' or x == '"' or x == '[' or x == ']' or x == '-':
            if str_p == "":
                continue
            else:
                contents.append(str_p)
                str_p = ""
        else:
            str_p = str_p + x
    articles_content.append(contents)

Dict_idf = {}
DictList = []
# 计算   x词，content文章
for content in articles_content:
    Dict_tf = {}
    for x in content:
        if not x in Dict_tf.keys():
            Dict_tf[x] = 1.0
            if not x in Dict_idf.keys():
                Dict_idf[x] = 1.0
            else:
                Dict_idf[x] += 1.0
        else:
            Dict_tf[x] += 1.0

    for k, v in Dict_tf.items():
        Dict_tf[k] = v / len(content)

    DictList.append(Dict_tf)

for k, v in Dict_idf.items():
    Dict_idf[k] = math.log(float(len(url_list)) / v)

for pos,x in enumerate(DictList):
    for k,v in x.items():
        DictList[pos][k] = v*Dict_idf[k]
    DictList[pos] = sorted(x.items(), key=lambda d: d[1], reverse=True)

# 写入文件
data = []
for pos in range(10):
    data2=[]
    data2.append("article_titile:")
    data2.append(articles_title[pos])
    data2.append([{"word": k,"value":round(v,4)} for k,v in DictList[pos][:10]])
    data.append(data2)

# Writing JSON data
with open('data.json', 'w') as f:
    json.dump(data, f)

[['CHANGCHUN', 'The', 'protection', 'zone', 'for', 'Siberian', 'tigers', 'in', 'Northeast', "China's", 'Jilin', 'province', 'will', 'be', 'expanded', 'according', 'to', 'an', 'ecological', 'protection', 'plan', 'by', 'the', 'local', 'government', 'According', 'to', 'the', 'plan', 'nine', 'new', 'protection', 'zones', 'covering', 'an', 'area', 'of', '390,000', 'hectares', 'will', 'be', 'built', 'in', 'the', 'province', 'By', '2020', 'an', '800,000', 'hectare', 'habitat', 'for', 'tigers', 'and', 'leopards', 'will', 'be', 'created', 'with', 'capacity', 'for', 'more', 'than', '30', 'fertile', 'tigers', 'and', 'over', '50', 'fertile', 'leopards', 'By', '2025', 'the', 'range', 'of', 'the', 'tigers', 'and', 'leopards', 'is', 'expected', 'to', 'reach', '2', 'million', 'hectares', 'The', 'regional', 'plan', 'is', 'in', 'line', 'with', 'the', 'Chinese', "government's", '13th', 'Five', 'Year', 'Plan', 'which', 'vows', 'to', 'strengthen', 'management', 'of', 'the', "country's", 'nature', 'reserves

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']